In [ ]:
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import fsspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import dask

In [ ]:
#lib for dask gateway
from dask_gateway import Gateway
from dask.distributed import Client
from dask import delayed
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=75)
client = Client(cluster)
cluster

In [ ]:
file_aws2 = 'https://mur-sst.s3.us-west-2.amazonaws.com/zarr'  #chunked for TS analysis
ds = xr.open_zarr(file_aws2,consolidated=True)
ds

In [ ]:
subset = ds.analysed_sst.sel(time=slice('2003-01-01','2019-12-31'),lat=slice(10,38),lon=slice(-121,-65))
subset = subset.chunk({'lat':200,'lon':200,'time':6209})
subset

In [ ]:
#subset[0,:,:].plot()

In [ ]:
%%time

with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    clim = subset.groupby('time.dayofyear').mean('time',keep_attrs=True)

#daily_clim=u.groupby('time.dayofyear').mean()
#sst_anom = sst.groupby('time.day')-clim  #take out annual mean to remove trends

#sst_anom = sst_anomaly.persist()


In [ ]:
clim = clim.chunk({'lat':2801,'lon':5601,'dayofyear':1})

In [ ]:
subset_clim = clim.sel(dayofyear=slice(223,260)) # we only need aug - 10th sept anomaly
subset_clim

In [ ]:
#subset_clim.load()

In [ ]:
subset_clim.to_netcdf('./../../data/gulf_2003_2019_climatology.nc')